In [3]:
# load pretrained model
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

Using cache found in /Users/jason/.cache/torch/hub/pytorch_vision_v0.6.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [5]:
# load an image from local file
from PIL import Image
from torchvision import transforms
input_image = Image.open("dog.jpg")

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with open('classes.txt') as f:
  labels = [line.strip() for line in f.readlines()]


with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print("output: ", output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# print(torch.nn.functional.softmax(output[0], dim=0))
_, index = torch.max(output, 1)
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100

_, indices = torch.sort(output, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]


[("208: 'Labrador retriever',", 60.26504135131836),
 ("173: 'Ibizan hound, Ibizan Podenco',", 9.754775047302246),
 ("178: 'Weimaraner',", 4.087791919708252),
 ("207: 'golden retriever',", 3.4103448390960693),
 ("176: 'Saluki, gazelle hound',", 2.2389512062072754)]

In [30]:
import torchvision
# define transformations
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_data = torchvision.datasets.ImageFolder(
    "./testimages",
    transform=transformations
)
dataloader = torch.utils.data.DataLoader(
    dataset=image_data,
    batch_size=32,
    shuffle=False,
    num_workers=8
)

counter = 0
# Tell torch not to calculate gradients
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()
device = torch.device("cpu")
criterion = torch.nn.NLLLoss()
# Set the optimizer function using torch.optim as optim library
optimizer = torch.optim.Adam(model.classifier.parameters())
with torch.no_grad():
    val_loss = 0
    accuracy = 0
    counter = 0
    for inputs, labels in dataloader:
        print(counter)
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass
        output = model.forward(inputs)
        # Calculate Loss
        valloss = criterion(output, labels)
        # Add loss to the validation set's running loss
        val_loss += valloss.item()*inputs.size(0)

        # Since our model outputs a LogSoftmax, find the real 
        # percentages by reversing the log function
        output = torch.exp(output)
        # Get the top class of the output
        top_p, top_class = output.topk(1, dim=1)
        # See how many of the classes were correct?
        equals = top_class == labels.view(*top_class.shape)
        print(top_class, labels.view(*top_class.shape))
        # Calculate the mean (get the accuracy for this batch)
        # and add it to the running accuracy for this epoch
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        # Print the progress of our evaluation
        counter += 1
        print(counter, "/", len(dataloader))

        # Get the average loss for the entire epoch
    valid_loss = val_loss/len(dataloader.dataset)
        # Print out the information
    print('Accuracy: ', accuracy/len(dataloader))
    # print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

Using cache found in /Users/jason/.cache/torch/hub/pytorch_vision_v0.6.0


0
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([[ 37],
        [817],
        [285],
        [ 79],
        [208],
        [319],
        [  1],
        [ 90],
        [ 71],
        [195]]) tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])
1 / 1
Accuracy:  0.0
